In [1]:
# dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
%matplotlib inline
# import dependencies
from getpass import getpass
import sqlalchemy
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, text
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# SQL Alchemy connection from RDS
secret = getpass('Enter the secret value: ')

args ={
    'host':"ogdataset.c11hekhsylui.us-west-1.rds.amazonaws.com",
    'port':'5432',
    'database':"og_dataset",
    'user':"attritionProject",
    'password':secret
}
engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{database}".format(**args))
connection = engine.connect()

Enter the secret value: ········


/Users/stephen/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# read in table from database
attrition_df = pd.read_sql('SELECT * FROM encoded_data', connection)
attrition_df

,Age,Attrition,Distance from Home,Monthly Income,Number Companies Worked,Percent Salary Hike,Total Working Years,Training Times Last Year,Years at Company,Years In Current Role,...,Stock Option Level_1,Stock Option Level_2,Stock Option Level_3,Education Level_1,Education Level_2,Education Level_3,Education Level_4,Education Level_5,Performance Rating_3,Performance Rating_4
0,41,1,1,5993,8,11,8,0,6,4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,49,0,8,5130,1,23,10,3,10,7,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,37,1,2,2090,6,15,7,3,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,33,0,3,2909,1,11,8,3,8,7,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,27,0,2,3468,9,12,6,3,2,2,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,0,23,2571,4,17,17,3,5,2,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1466,39,0,6,9991,4,15,9,5,7,7,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1467,27,0,4,6142,1,20,6,0,6,2,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1468,49,0,2,5390,2,14,17,3,9,6,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [4]:
# check version number
import imblearn
print(imblearn.__version__)

0.7.0


In [5]:
# Split the preprocessed data into a training and testing dataset
# Split our preprocessed data into our features and target arrays
X = attrition_df.drop("Attrition",1).values
y = attrition_df["Attrition"].values
X_traino, X_testo, y_traino, y_testo = train_test_split(X, y, random_state=78, test_size= 0.20, train_size= 0.80, stratify =y)

/Users/stephen/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler# define oversampling strategy
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_traino, y_traino)
# summarize class distribution
print(Counter(y_over))

Counter({0: 986, 1: 986})


In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scalero = scaler.fit(X_over)

# Scale the data
X_train_scaledo = X_scalero.transform(X_over)
X_test_scaledo = X_scalero.transform(X_testo)

In [15]:
Foresto = RandomForestClassifier(random_state=77, n_estimators=5000)
Foresto.fit(X_over, y_over)
print(f'Training Score: {Foresto.score(X_train_scaledo, y_over)}')
print(f'Testing Score: {Foresto.score(X_test_scaledo, y_testo)}')

Training Score: 0.8179513184584178
Testing Score: 0.8707482993197279


In [16]:
y_trueo = y_testo
y_pred_testo = Foresto.predict(X_testo)
#y_pred_testo

In [17]:
confusion_matrix(y_trueo, y_pred_testo)

array([[243,   4],
       [ 36,  11]])

In [18]:
accuracy_score(y_trueo, y_pred_testo)

0.8639455782312925

In [19]:
print(classification_report(y_trueo, y_pred_testo))

              precision    recall  f1-score   support

           0       0.87      0.98      0.92       247
           1       0.73      0.23      0.35        47

    accuracy                           0.86       294
   macro avg       0.80      0.61      0.64       294
weighted avg       0.85      0.86      0.83       294



In [20]:
# yes/no in y_testo
never =0
ever=0
for k in y_trueo:
    if k == '0':
        never=never+1
    if k== '1':
        ever=ever+1
print(f'yes {ever}, no {never}')

yes 0, no 0


In [21]:
# yes/no that it predicts
hever =0
lever=0
for k in y_pred_testo:
    if k == '0':
        hever=hever+1
    if k== '1':
        lever=lever+1
print(f'yes {lever}, no {hever}')

yes 0, no 0
